# Implementação da Parte 1 do Trabalho

Essa é a nossa implementação da primeira parte do trabalho de MC322. Nela iremos implementar as intefaces `IEnquirer`, `IResponder`, `IDataSource`, `ITableProducer` (dadas pelo professor) e seus respectivos receptaculos, em classes chamadas `Doctor`, `Patient`(criadas pelo grupo) e `DataSet` (criada pelo professor).

`Doctor` deve ser capaz de interagir com um `Patient` e, através de perguntas, ser capaz de diagnosticar a doença do paciente.

`Patient` deve ser capaz de sortear uma doença da lista armazenada no banco de doenças, e responder ao Doutor para que tal seje capaz de diagnosticar sua doença.

## Interfaces relacionadas ao `DataSet`

Essas interfaces serão implementadas pela componente `DataSet`, ou fazem a conexão dessa componente com outras componentes.

In [14]:
//Essa interface define os metodos usados para definir o Data Source
public interface IDataSource {
  public String getDataSource();
  public void setDataSource(String dataSource);
}

//Essa interface define os metodos que geram a tabela de atributos e instancias do Data Set
public interface ITableProducer {
  String[] requestAttributes();
  String[][] requestInstances();
}

//Essa interface faz a conexao com a interface ITableProducer
public interface ITableProducerReceptacle {
  public void connect(ITableProducer producer);
}

//Combinando as interfaces do DataSet em IDataSet
public interface IDataSet extends IDataSource, ITableProducer {
}

## Componente `DataSet`

Essa componente implementa e configura um DataSet que contém as doenças e seus sintomas

In [15]:
//importando as bibliotecas usadas pelo DataSet
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.util.ArrayList;


public class DataSetComponent implements IDataSet {
  private String dataSource = null;
  private String[] attributes = null;
  private String[][] instances = null;
  
  public DataSetComponent() {
    /* nothing */
  }

  public String getDataSource() {
    return dataSource;
  }

  public void setDataSource(String dataSource) {
    this.dataSource = dataSource;
    if (dataSource == null) {
      attributes = null;
      instances = null;
    } else
      readDS();
  }
  
  public String[] requestAttributes() {
    return attributes;
  }
  
  public String[][] requestInstances() {
    return instances;
  }
  
  private void readDS() {
    ArrayList<String[]> instArray = new ArrayList<String[]>();
    try {
      BufferedReader file = new BufferedReader(new FileReader(dataSource));
        
      String line = file.readLine();
      if (line != null) {
        attributes = line.split(",");
        line = file.readLine();
        while (line != null) {
          String[] instLine = line.split(",");
          instArray.add(instLine);
          line = file.readLine();
        }
        instances = instArray.toArray(new String[0][]);
      }
        
      file.close();
    } catch (IOException erro) {
      erro.printStackTrace();
    }
  }
  
}

## Interfaces relacionadas ao `Patient`

Essas interfaces serão implementadas pela componente `Patient`, ou fazem a conexão dessa componente com outras componentes.

In [16]:
//Essa interface define os metodos usados para respostas
public interface IResponder {
    public String ask(String question);
    public boolean finalAnswer(String answer);
}

//Essa interface faz a conexao com a interface IResponder
public interface IResponderReceptacle {
    public void connect(IResponder responder);
}

//Combinando as interfaces do Patient em IPatient
public interface IPatient extends IResponder, ITableProducerReceptacle {
}

## Componente `Patient`

Essa componente implementará os métodos de `IPatient` com a finalidade de armazenar uma doença aleatória e seus sintomas. Ele será conectado à componente `Doctor` (que irá diagnosticar esse paciente) e se conectará à componente `DataSet` (que armazena as possíveis doenças e seus sintomas).

In [17]:
//importando a biblioteca de numeros aleatorios
import java.util.Random;

public class Patient implements IPatient{
    private String[]   gabarito;
    private String[]   attributes;
    private ITableProducer ds;
    private int doenca;
    private Random rand;
    
    public Patient(ITableProducer producer){
        connect(producer);
        rand = new Random();
        attributes = ds.requestAttributes();
        doenca = rand.nextInt(attributes.length);
        gabarito = ds.requestInstances()[doenca];
    }
    
    public String ask(String question){
        for(int i = 0; i < attributes.length; i++)
            if(question == attributes[i])
                return gabarito[i];
        return "unknown";
    }
    
    public boolean finalAnswer(String answer){
        if(gabarito[gabarito.length-1].equals(answer))
            return true;
        else
            return false;
    }
    
    public void connect(ITableProducer producer){
        ds = producer;
    }
}

## Interfaces relacionadas ao `Doctor`

Essas interfaces serão implementadas pela componente `Doctor`.

In [18]:
//Essa interface define os metodos usados para iniciar o diagnostico
public interface IEnquirer {
    public void startInterview();
}

//Combinando as interfaces do Doctor em IDoctor
public interface IDoctor extends IEnquirer, IResponderReceptacle, ITableProducerReceptacle {
}

## Componente `Doctor`

Essa componente implementará os métodos de `IDoctor` com a finalidade de diagnosticar uma doença de um paciente. Ele componente se conecta à componente `Patient` (para diagnosticar sua doença) e à componente `DataSet` (para gerar o diagnóstico) 

In [35]:
import java.util.Random;

// combinando duas interfaces para IPatient
public interface IDoctor extends IEnquirer, IResponderReceptacle, ITableProducerReceptacle {
}

public class Doctor implements IDoctor {
    private int patientN = 0;

    private ITableProducer producer;
    private IResponder responder;
  
    public void connect(ITableProducer producer) {
        this.producer = producer;
    }
    
    public void connect(IResponder responder) {
        this.responder = responder;
    }
        
    public void startInterview() {
        String attributes[] = producer.requestAttributes();
        String instances[][] = producer.requestInstances();
        
        String sintomas[] = new String[attributes.length];

        for (int a = 0; a < attributes.length - 1; a++){
            sintomas[a] = responder.ask(attributes[a]);
            System.out.println("Question: " + sintomas[a]);
        }
        
        for (int i=0; i < instances.length; i++){
            for (int a=0; a < attributes.length-1; a++){
                if (!sintomas[a].equals(instances[i][a]))
                    break;
                
                else{
                    if (a == attributes.length-2){
                        System.out.println("Disease guess: " + instances[i][attributes.length - 1]);
                        boolean result = responder.finalAnswer(instances[i][attributes.length - 1]);
                        System.out.println("Result: " + (result ? "I am right =)" : "I am wrong =("));
                        i = instances.length;
                        break;
                    }
                }
            }
        }
    }
    
}

// instanciando o componente DataSet
IDataSet dataset = new DataSetComponent();
dataset.setDataSource("roteiros-auxiliares/db/zombie-health-new-cases20.csv");

// instanciando o componente paciente
IPatient aPatient = new Patient(dataset);

// conectando-o no componente DataSet
aPatient.connect(dataset);

// instanciando o componente doutor louco
IDoctor cDoctor = new Doctor();

// conectando-o ao componente DataSet
cDoctor.connect(dataset);

// conectando-o ao componente paciente
cDoctor.connect(aPatient);

// executando a entrevista
cDoctor.startInterview();



Question: 0
Question: 1
Question: 0
Question: 1
Question: 0
Question: 0
Question: 0
Question: 1
Disease guess: Infecção Bacteriana
Result: I am right =)
